In [144]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [159]:
data = pd.read_csv('temp/因子数据初提取.csv',encoding = 'GBK',engine = 'python')

In [160]:
# 将需要预测的数据提取并保存
data_pred = data[data['年份（年末）'] == 7]
# 重置索引
data_pred.reset_index(inplace = True)
data_pred.drop(axis = 0,columns = 'index', inplace = True)
data_pred.to_csv('需要预测的第七年数据.csv',index = False,encoding = 'GBK')
del data_pred

# 删除非用于建模的数据
data.dropna(axis = 0,subset = ['近两年送转比例平均','次年是否送转'],inplace = True)
# 重置索引
data.reset_index(inplace = True)
data.drop(axis = 0,columns = 'index', inplace = True)

In [161]:
data.isnull().sum()

股票编号                       0
年份（年末）                     0
总股本                        0
每股资本公积                    62
每股未分配利润                    4
基本每股收益                     0
基本每股收益同比增长(%)              2
每股净资产                      0
每股净资产相对年初增长(%)           295
每股现金流                      1
每股营业收入                     0
营业收入同比增速(%)                4
加权平均净资产收益率                64
摊薄净资产收益率同比增长(%)          123
净利润同比增长                    0
上市年限                       0
交通运输、仓储和邮政业                0
住宿和餐饮业                     0
信息传输、软件和信息技术服务业            0
农、林、牧、渔业                   0
制造业                        0
卫生和社会工作                    0
建筑业                        0
房地产业                       0
批发和零售业                     0
教育                         0
文化、体育和娱乐业                  0
水利、环境和公共设施管理业              0
电力、热力、燃气及水生产和供应业           0
科学研究和技术服务业                 0
租赁和商务服务业                   0
综合                         0
采矿业                        0
金融业                        0
是否次新股         

'每股净资产相对年初增长(%)'，摊薄净资产收益率同比增长(%)缺失值相对较多

In [166]:
# 缺失值处理
for i in data.columns[3:-2]:
    for j in range(len(data)):
        if (data[i].isnull())[j]: # 若为空值，则用该年份该因子的平均值填充
            year = data.iloc[j]['年份（年末）']
            avg = np.mean(data[data['年份（年末）'] == year][i])
            data[i][j] = avg
            #data[i][j] = np.mean(data[i])

为避免数据中极端值对结果产生过多影响，使用“3𝜎准则 ”对数据进行处理，将超过上下限的极端值用上下限值代替。 

In [169]:
# 对比率性数据极端值处理
for i in data.columns[3:15]:
    mean = data.describe()[i]['mean']
    std = data.describe()[i]['std']
    #print(i)
    #print('mean:%.2f'%mean)
    #print('std:%.2f'%std)
    for j in range(len(data)):
        if data[i][j] > mean + 3*std:
            data[i][j] = mean + 3*std
        elif data[i][j] < mean - 3*std:
            data[i][j] = mean - 3*std

In [174]:
data.describe()

,股票编号,年份（年末）,总股本,每股资本公积,每股未分配利润,基本每股收益,基本每股收益同比增长(%),每股净资产,每股净资产相对年初增长(%),每股现金流,...,金融业,是否次新股,是否小盘成长,是否有行业龙头概念,股价（年末最后20个交易日收盘价加权平均）,股价年化波动率,本年送转比例,近两年送转比例平均,次年是否送转,次年是否高送转
count,13313.000000,13313.000000,1.331300e+04,13313.000000,13313.000000,13313.000000,13313.000000,13313.000000,13313.000000,13313.000000,...,13313.000000,13313.0,13313.000000,13313.000000,13313.000000,13313.000000,13313.000000,13313.000000,13313.000000,13313.000000
mean,1740.324946,4.112597,1.695949e+09,1.931929,1.220454,0.291902,-41.966542,4.458641,13.492076,0.162813,...,0.022985,0.0,0.036055,0.144070,11.435230,0.470925,0.249602,0.249514,0.734620,0.154886
std,997.266524,1.424329,1.289589e+10,1.719902,1.492513,0.423285,516.074091,2.773390,187.837358,1.026379,...,0.149861,0.0,0.186434,0.351174,10.403883,0.175497,0.434488,0.330880,0.441552,0.361810
min,1.000000,2.000000,1.388889e+07,-2.883300,-4.588097,-1.355541,-4368.177065,-8.043787,-4412.526424,-4.834583,...,0.000000,0.0,0.000000,0.000000,0.373333,0.000000,0.000000,0.000000,0.000000,0.000000
25%,880.000000,3.000000,2.470550e+08,0.708700,0.473200,0.075900,-45.070400,2.591900,-10.065500,-0.234400,...,0.000000,0.0,0.000000,0.000000,5.222087,0.359215,0.000000,0.018000,0.000000,0.000000
50%,1729.000000,4.000000,4.577847e+08,1.493100,1.024000,0.230000,-5.814000,3.885900,3.415700,0.027800,...,0.000000,0.0,0.000000,0.000000,8.544010,0.428849,0.070000,0.100000,1.000000,0.000000
75%,2608.000000,5.000000,9.226023e+08,2.656600,1.774000,0.459100,28.571400,5.686600,12.082600,0.363700,...,0.000000,0.0,0.000000,0.000000,14.308175,0.539036,0.250000,0.400000,1.000000,0.000000
max,3466.000000,6.000000,3.564070e+11,9.306521,7.114903,1.952242,4271.771457,17.122948,4477.051490,5.215214,...,1.000000,0.0,1.000000,1.000000,314.208539,3.000381,6.787000,6.479000,1.000000,1.000000


In [176]:
data.to_csv('temp/预处理因子数据.csv',index = False, encoding = 'GBK')